In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
batchsize=8

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory('/content/drive/MyDrive/Project-DDD3/Dataset_Prepared/Dataset_New/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory('/content/drive/MyDrive/Project-DDD3/Dataset_Prepared/Dataset_New/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 42818 images belonging to 2 classes.
Found 10703 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory('/content/drive/MyDrive/Project-DDD3/Dataset_Prepared/Dataset_New/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 13379 images belonging to 2 classes.


In [ ]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
bmodel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalizati

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalization[0][0

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/models/model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=10)

<ipython-input-11-e7034fc89d2a>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/10
5352/5352 [==============================] - ETA: 0s - loss: 0.1853 - accuracy: 0.9292
Epoch 1: val_loss improved from inf to 0.30318, saving model to /content/drive/MyDrive/models/model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5352/5352 [==============================] - 12011s 2s/step - loss: 0.1853 - accuracy: 0.9292 - val_loss: 0.3032 - val_accuracy: 0.8824 - lr: 0.0010
Epoch 2/10
5352/5352 [==============================] - ETA: 0s - loss: 0.1551 - accuracy: 0.9404
Epoch 2: val_loss improved from 0.30318 to 0.28275, saving model to /content/drive/MyDrive/models/model.h5
5352/5352 [==============================] - 937s 175ms/step - loss: 0.1551 - accuracy: 0.9404 - val_loss: 0.2828 - val_accuracy: 0.8900 - lr: 0.0010
Epoch 3/10
5352/5352 [==============================] - ETA: 0s - loss: 0.1519 - accuracy: 0.9427
Epoch 3: val_loss improved from 0.28275 to 0.26531, saving model to /content/drive/MyDrive/models/model.h5
5352/5352 [==============================] - 935s 175ms/step - loss: 0.1519 - accuracy: 0.9427 - val_loss: 0.2653 - val_accuracy: 0.9031 - lr: 0.0010
Epoch 4/10
 440/5352 [=>............................] - ETA: 11:03 - loss: 0.1371 - accuracy: 0.9486

UnknownError: ignored

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

<ipython-input-25-87a1e491e135>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.1295931488275528
0.9521462917327881


In [ ]:
# acc_vr, loss_vr = model.evaluate_generator(validation_data)
# print(acc_vr)
# print(loss_vr)

<ipython-input-26-857f2865b528>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.24890804290771484
0.9062879681587219


In [ ]:
# acc_test, loss_test = model.evaluate_generator(test_data)
# print(acc_tr)
# print(loss_tr)

<ipython-input-27-229e55840385>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.1295931488275528
0.9521462917327881
